In [3]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from difflib import SequenceMatcher

In [2]:
original_code = """
int main()
 {
     int a, b = 0;
     print("Hello");
     return 0;
 }"""

changed_code = """
int main()
 {
     int a, b = 0;
     print("Helo");
     return 0;
 }"""

original_short = "ehello"
changed_short = "helio"

original_array = ["int", "i", "=", "5", ";"]
changed_array = ["\n", "int", "i", "=", "4", ";"]

In [3]:
seq = SequenceMatcher(None, original_code, changed_code)

In [4]:
seq.get_opcodes()

[('equal', 0, 49, 0, 49),
 ('delete', 49, 50, 49, 49),
 ('equal', 50, 72, 49, 71)]

In [5]:
seq_short = SequenceMatcher(None, original_short, changed_short)

In [6]:
seq_short.get_opcodes()

[('delete', 0, 1, 0, 0),
 ('equal', 1, 4, 0, 3),
 ('replace', 4, 5, 3, 4),
 ('equal', 5, 6, 4, 5)]

In [7]:
np.zeros_like(list(original_short), dtype=np.int32)[3:3]

array([], dtype=int32)

In [8]:
seq = SequenceMatcher(None, original_array, changed_array)

In [9]:
seq.get_opcodes()

[('insert', 0, 0, 0, 1),
 ('equal', 0, 3, 1, 4),
 ('replace', 3, 4, 4, 5),
 ('equal', 4, 5, 5, 6)]

## Combining metadata files

In [31]:
import shutil
import glob

def merge_metadata_files():
    output_filename = 'full_metadata.csv'
    column_name_written = False

    with open(output_filename, 'wb') as output:
        for file_name in glob.glob('D:\\defence_data\\Project_CodeNet\\metadata\\*.csv'):
            if file_name != output_filename:
                with open(file_name, 'rb') as current:
                    content = current.readlines()
                    
                    if not column_name_written:
                        output.writelines([content[0]])
                        column_name_written = True
                    
                    for i in range(len(content)):      
                        if(str(content[i]).split(',')[4] == "C"):
                            output.writelines([content[i]])

In [32]:
merge_metadata_files()

## Extract data

In [33]:
metadata = pd.read_csv("full_metadata.csv")
metadata.sample(10)

C:\Users\Hp\AppData\Local\Temp\ipykernel_18036\362672389.py:1: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  metadata = pd.read_csv("full_metadata.csv")


,submission_id,problem_id,user_id,date,language,original_language,filename_ext,status,cpu_time,memory,code_size,accuracy
650143,s567951058,p03339,u993046043,1549906690,C,C (GCC 5.4.1),c,Accepted,5.0,2816.0,650,NaN
745176,s496284215,p03998,u014442925,1554225157,C,C (GCC 5.4.1),c,Accepted,1.0,128.0,1834,NaN
581591,s210305587,p03053,u974706915,1563825829,C,C (GCC 5.4.1),c,Time Limit Exceeded,1055.0,2048.0,943,NaN
568041,s382643300,p03000,u896734414,1560712479,C,C (GCC 5.4.1),c,Wrong Answer,1.0,128.0,397,NaN
308099,s729260616,p02402,u333124620,1495699278,C,C,c,Accepted,0.0,640.0,322,20/20
478554,s510002348,p02727,u865456985,1585687155,C,C (GCC 5.4.1),c,Wrong Answer,61.0,1280.0,1282,NaN
12718,s819795061,p00003,u862220432,1497717311,C,C,c,Wrong Answer,0.0,580.0,353,0/1
612082,s658425991,p03177,u657913472,1548218129,C,C (GCC 5.4.1),c,Accepted,157.0,256.0,300,NaN
133188,s415110111,p02256,u501711118,1428629819,C,C,c,Accepted,0.0,592.0,218,10/10
750050,s533580165,p04030,u721060623,1491090820,C,C (GCC 5.4.1),c,Accepted,1.0,128.0,304,NaN


In [34]:
metadata.shape

(754058, 12)

In [40]:
def read_code_file(problem_id, submission_id):
    with open("D:\\defence_data\\Project_CodeNet\\data\\{}\\C\\{}.c"
              .format(problem_id, submission_id),encoding="utf8") as f:
        return f.read()

In [41]:
filtered_df = pd.DataFrame()

In [42]:
filtered_df

""


In [43]:
metadata.index.size

754058

In [44]:
count = 0
max_difference = 30
indexes = metadata.index

for i in indexes[0:-1]:
    if(metadata["status"][i] == "Runtime Error" and metadata["status"][i+1] == "Accepted"
      and metadata["user_id"][i] == metadata["user_id"][i+1]
      and abs(metadata["code_size"][i] - metadata["code_size"][i+1]) < max_difference
      and metadata["problem_id"][i] == metadata["problem_id"][i+1]):
        
        problem_id = metadata["problem_id"][i]
        
        bug_submission_id = metadata["submission_id"][i]
        correct_submission_id = metadata["submission_id"][i+1]
        
        filtered_df = pd.concat([filtered_df, pd.DataFrame(
            {
                "buggy_code": read_code_file(problem_id, bug_submission_id),
                "correct_code": read_code_file(problem_id, correct_submission_id),
                "problem_id": problem_id,
                "buggy_code_submission_id": bug_submission_id,
                "correct_code_submission_id": correct_submission_id
            },
            index=[count])])
        count += 1
        
print(count)

3829


In [45]:
filtered_df

,buggy_code,correct_code,problem_id,buggy_code_submission_id,correct_code_submission_id
0,"i;main(j){for(;i++<9;)for(j=0;j++<9;)printf(""%...","i;main(j){for(;i++<9;)for(j=0;j++<9;)printf(""%...",p00000,s807257751,s380297172
1,#include <stdio.h>\n\nint main()\n{\n int i...,#include <stdio.h>\n\nint main()\n{\n int i...,p00000,s614489951,s192028699
2,"int i,j;main(){for(i;++i<10;){for(j=0;++j<10;)...","int i,j;main(){for(i;++i<10;){for(j=0;++j<10;)...",p00000,s187894075,s855404770
3,"main(i,j){for(i=1;i<=9;i++)for(j=1;j<=9;j++)pr...","main(i,j){for(i=1;i<=9;i++)for(j=1;j<=9;j++)pr...",p00000,s164749481,s229822026
4,"main(i,j){for(i=1;i<=9;i++)for(j=1;j<=9;printf...","main(i,j){for(j=1;j<=9;printf(""%dx%d=%d\n"",i,j...",p00000,s788910072,s525142539
...,...,...,...,...,...
3824,"#include <stdio.h>\n\nint main(){\n int a,b...","#include <stdio.h>\n\nint main(){\n int a,b...",p03997,s359521643,s068836925
3825,#include<stdio.h>\n#include<string.h>\n\nint m...,"#include<stdio.h>\nint main()\n{\n char c1,...",p04012,s413335847,s086128679
3826,#include<stdio.h>\n#include<stdlib.h>\n#includ...,#include<stdio.h>\n#include<stdlib.h>\n#includ...,p04013,s583188520,s830816697
3827,#include<stdio.h>\n#include<string.h>\n\n#defi...,#include<stdio.h>\n#include<string.h>\n\n#defi...,p04044,s175360180,s701954551


In [46]:
train, test = train_test_split(filtered_df, test_size=0.2)

In [47]:
train

,buggy_code,correct_code,problem_id,buggy_code_submission_id,correct_code_submission_id
1283,"c;main(r){for(;scanf(""%d%d"",&r,&c),r;)puts(r&c...","c;main(r){for(;scanf(""%d%d"",&r,&c),r;)puts((r&...",p00657,s362273130,s011867637
845,"a[100],l,r,p,n;main(c){for(;l=scanf(""%d"",&r)*r...","a[100],l,r,p,n;main(c){for(;l=scanf(""%d"",&r)*r...",p00188,s446966487,s278308646
813,"#include <stdio.h>\nint main (void)\n{\nint a,...","#include <stdio.h>\nint main (void)\n{\nint a,...",p00161,s570559021,s490040394
532,#include <stdio.h>\n#include <math.h>\n\nchar ...,#include <stdio.h>\n#include <math.h>\n\nchar ...,p00044,s367988925,s836296672
2965,#include <stdio.h>\nint main(void)\n{\n int a...,#include <stdio.h>\nint main(void)\n{\n int a...,p02398,s272396205,s183587626
...,...,...,...,...,...
1350,"Y,M,D;main(n){for(scanf(""%d"",&n);Y=n--;scanf(""...","main(Y,M,D){for(scanf(""%*d"");~scanf(""%d%d%d"",&...",p00760,s689276410,s312350405
3703,#include <stdio.h>\n\nint main(void){\n\n i...,#include <stdio.h>\n\nint main(void){\n\n i...,p02755,s606604490,s226498916
2252,"#include<stdio.h>\n \nvoid swap(int* a,int* b)...","#include<stdio.h>\n \nvoid swap(int* a,int* b)...",p02276,s948095782,s563397808
3550,"h;main(w){for(;~scanf(""%d%d"",&h,&w);puts(""""))f...","h;main(w){for(;scanf(""%d%d"",&h,&w),h|w;puts(""""...",p02494,s863615576,s020921668


In [48]:
train.to_csv("filtered_data_train.csv", index=False)
test.to_csv("filtered_data_test.csv", index=False)

## Masking and Tokenization

In [4]:
df_train = pd.read_csv("filtered_data_train.csv")
df_train.sample(5)

,buggy_code,correct_code,problem_id,buggy_code_submission_id,correct_code_submission_id
245,#include<stdio.h>\nint main(void)\n{\n int ...,#include<stdio.h>\nint main(void)\n{\n int ...,p00489,s355798126,s458763493
2079,#include <stdio.h>\n#include<string.h>\nint bi...,#include <stdio.h>\n#include<string.h>\nint bi...,p02268,s652144487,s074007499
2341,#include<stdio.h>\n\nint main()\n{\n int x...,#include<stdio.h>\n\nint main()\n{\n int x...,p02480,s312665934,s574762356
20,#include<stdio.h>\nint main (void)\n{\n\tint x...,#include<stdio.h>\nint main (void)\n{\n\tint x...,p02388,s704739223,s990980705
2952,"#include<stdio.h>\nint main(){\n int a, b, ...","#include<stdio.h>\nint main(){\n int a, b, ...",p02392,s629761874,s421007876


In [5]:
df_test = pd.read_csv("filtered_data_test.csv")
df_test.sample(5)

,buggy_code,correct_code,problem_id,buggy_code_submission_id,correct_code_submission_id
159,\n#include <stdio.h>\nint main(void)\n{\n\tint...,\n#include <stdio.h>\nint main(void)\n{\n\tint...,p00049,s374071296,s452492909
367,#include<stdio.h>\n#define N 10\n#define M 11\...,#include<stdio.h>\n#define N 100\n#define M 11...,p00102,s245839272,s703478245
263,#include <stdio.h>\n#include <stdlib.h>\n#incl...,#include <stdio.h>\n#include <stdlib.h>\n#incl...,p02265,s938007148,s737593613
341,"#include<stdio.h>\n\nint main(void){\n\nint a,...","#include<stdio.h>\n\nint main(void){\n\nint a,...",p02399,s068373930,s972920950
675,#include <stdio.h>\n#include <limits.h>\n\n#de...,#include <stdio.h>\n#include <limits.h>\n\n#de...,p00586,s848573407,s834614614


In [12]:
df_train['buggy_code'].value_counts

<bound method IndexOpsMixin.value_counts of 0       c;main(r){for(;scanf("%d%d",&r,&c),r;)puts(r&c...
1       a[100],l,r,p,n;main(c){for(;l=scanf("%d",&r)*r...
2       #include <stdio.h>\nint main (void)\n{\nint a,...
3       #include <stdio.h>\n#include <math.h>\n\nchar ...
4       #include <stdio.h>\nint main(void)\n{\n  int a...
                              ...                        
3058    Y,M,D;main(n){for(scanf("%d",&n);Y=n--;scanf("...
3059    #include <stdio.h>\n\nint main(void){\n\n    i...
3060    #include<stdio.h>\n \nvoid swap(int* a,int* b)...
3061    h;main(w){for(;~scanf("%d%d",&h,&w);puts(""))f...
3062    #include<stdio.h>\n#define N 100\n#define NIL ...
Name: buggy_code, Length: 3063, dtype: object>

In [9]:
df_test['correct_code'][2]

'#include<stdio.h>\n\nvoid CountingSort(int *, int *, int);\n\nint C[10000];\n\nint main(){\n\n  int i, n;\n\n  scanf("%d", &n);\n\n  int A[n], B[n];\n    \n  for(i = 0; i < n; i++){\n    scanf("%d", &A[i]);\n  }\n\n  CountingSort(A, B, n);\n\n  for(i = 1; i < n; i++){\n    printf("%d ", B[i]);\n  }\n  printf("%d\\n", B[i]);\n\n\n  return 0;\n}\n\nvoid CountingSort(int *A, int *B, int n){\n\n  int i;\n  \n  for(i = 0; i < 10000; i++){\n    C[i] = 0;\n  }\n\n  /* C[i] に i の出現数を記録する */\n  for(i = 0; i < n; i++){\n    C[A[i]]++;\n  }\n\n  /* C[i] に i 以下の数の出現数を記録する*/\n  for(i = 1; i <= 10000; i++){\n    C[i] = C[i] + C[i-1];\n  }\n  \n  for(i = n-1; i >= 0; i--){ \n    B[C[A[i]]] = A[i];\n    C[A[i]]--;\n  }\n}      \n\n'